In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import re, ast
#change to appropriate home folder
folder = ''
import warnings
warnings.filterwarnings('ignore')
type_str = 'test'

In [2]:
#first try gpt3.5-turbo. change to appropriate apikey.
openai_api = ''
import os
import openai
from openai import OpenAI
openai.organization = ""
openai.api_key = openai_api


In [9]:
drug_label_text = pd.read_csv(folder+'{}_drug_label_text.csv'.format(type_str))
drug_label_text.head(1)

,drug_name,section_name,section_text
0,IMPAVIDO,adverse reactions,6 ADVERSE REACTIONS\n\n Because clinical ...


In [10]:
#read in meddra vocabulary, and make dictionary
meddra_df = pd.read_csv(folder+'umls_meddra_en.csv')
meddra_df = meddra_df[['TTY', 'CODE', 'STR']].drop_duplicates()
meddra_df = meddra_df[meddra_df.TTY.isin(['PT', 'LLT'])]
meddra_df['STR'] = meddra_df.STR.apply(lambda x: x.lower())
meddra_df['LEN'] = meddra_df.STR.apply(lambda x: len(x))
meddra_df = meddra_df[meddra_df.LEN >= 5]
meddra_dict = dict(zip(meddra_df.STR, meddra_df.CODE))
print(meddra_df.shape[0])
meddra_df.head(1)

100897


,TTY,CODE,STR,LEN
0,PT,10000647,acute abdomen,13


In [11]:
found_ades = []
meddra_names = meddra_df.STR.tolist()
for ade_text in tqdm(drug_label_text.section_text.tolist()):
  ar_text = ' '.join(ade_text.split()).lower()
  found_terms = []
  for concept_name in meddra_names:
    if ar_text.find(concept_name) == -1:
      continue
    else:
      found_terms.append(concept_name)
  found_ades.append(found_terms)

100%|██████████| 237/237 [01:09<00:00,  3.43it/s]


In [12]:
drug_label_potential_df = drug_label_text.copy()
drug_label_potential_df['str_match'] = [list(set(i)) for i in found_ades]
gpt35_output = pd.read_csv(folder+'{}_drug_label_gpt35_output.csv'.format(type_str))
drug_label_potential_df = drug_label_potential_df.merge(gpt35_output, on=['drug_name', 'section_name'], how = 'left')
gpt4_output = pd.read_csv(folder+'{}_drug_label_gpt4_output.csv'.format(type_str))
drug_label_potential_df = drug_label_potential_df.merge(gpt4_output, on=['drug_name', 'section_name'], how = 'left')
drug_label_potential_df.to_csv(folder+'{}_drug_label_text_potential_terms.csv'.format(type_str), index=False)
drug_label_potential_df.head()

,drug_name,section_name,section_text,str_match,gpt35_output,gpt4_output
0,IMPAVIDO,adverse reactions,6 ADVERSE REACTIONS\n\n Because clinical ...,"[asthenia, dizziness, transaminases, swelling,...",The adverse drug event terms in the given text...,NaN
1,IMPAVIDO,boxed warnings,\n\n BOXED WARNING: WARNING: EMBRYO-FETAL T...,"[pregnancy test, pregnancy, pregnant, death, t...","fetal harm, teratogenicity, pregnancy test, re...","EMBRYO-FETAL TOXICITY, fetal harm, Fetal death..."
2,IMPAVIDO,warnings and precautions,5 WARNINGS AND PRECAUTIONS\n\n\n\n EXCER...,"[transaminases, pregnant, leishmaniasis, cytop...","Embryo-Fetal Toxicity, Reproductive effects, R...","Embryo-Fetal Toxicity, Reproductive effects, R..."
3,LIVALO,adverse reactions,6 ADVERSE REACTIONS\n\n The following ser...,"[asthenia, dizziness, cognitive impairment, tr...","Rhabdomyolysis, myoglobinuria, acute renal fai...","Rhabdomyolysis, myoglobinuria, acute renal fai..."
4,LIVALO,warnings and precautions,5 WARNINGS AND PRECAUTIONS\n\n\n\n EXCER...,"[transaminases, muscle pain, trauma, rhabdomyo...","myopathy, rhabdomyolysis, hepatic transaminase...","Skeletal muscle effects, myopathy, rhabdomyoly..."


In [4]:
drug_label_potential_df = pd.read_csv(folder+'{}_drug_label_text_potential_terms.csv'.format(type_str))
drug_label_potential_df['str_match'] = drug_label_potential_df['str_match'].apply(lambda x: ast.literal_eval(x))
drug_label_potential_df['str_match_num'] = drug_label_potential_df['str_match'].apply(lambda x: len(x))

In [5]:
drug_label_potential_df['str_match_num'].describe()

count    237.000000
mean      47.704641
std       38.882119
min        0.000000
25%       18.000000
50%       40.000000
75%       67.000000
max      229.000000
Name: str_match_num, dtype: float64